In [1]:
import random

In [2]:
# initialization

max_vol = 4

In [ ]:
bin_text = """
yellow-orange-gray-lg
lb-lg-purple-red
purple-lg-pink-dg
brown-red-pink-yellow
purple-brown-db-lg
yellow-purple-gray-lg
dg-lb-gray-orange
pink-lb-db-red
lg-lb-db-db
orange-pink-brown-dg
orange-yellow-lg-gray
db-dg-brown-red"""

# bin_text = """
# red-red-lg-orange
# orange-lb-lb-lg
# db-db-red-orange
# lb-orange-pink-lb
# gray-gray-db-gray
# gray-red-lg-pink
# pink-db-lg-pink"""

bins = translate_bin_text(bin_text)
print(bins)
try_options(bins)

['yogi', 'liur', 'uipa', 'wrpy', 'uwdi', 'yugi', 'algo', 'pldr', 'ildd', 'opwa', 'oyig', 'dawr', '', '']
search depth: 0
search depth: 1
search depth: 2
search depth: 3
search depth: 4
search depth: 5
search depth: 6
search depth: 7
search depth: 8
search depth: 9
search depth: 10
search depth: 11
search depth: 12
search depth: 13
search depth: 14
search depth: 15
search depth: 16
search depth: 17
search depth: 18
search depth: 19
search depth: 20
search depth: 21
search depth: 22
search depth: 23
search depth: 24
search depth: 25
search depth: 26
search depth: 27
search depth: 28
search depth: 29
search depth: 30
search depth: 31
search depth: 32
search depth: 33
search depth: 34
search depth: 35
search depth: 36
search depth: 37
search depth: 38


In [3]:
color_code = {
    'lb': 'l',
    'db': 'd',
    'lg': 'i',
    'dg': 'a',
    'orange': 'o',
    'red': 'r',
    'purple': 'u',
    'pink': 'p',
    'gray': 'g',
    'yellow': 'y',
    'brown': 'w'
}

def translate_bin_text(bin_text):
    bins = []
    
    tubes = bin_text.split()
    for tube in tubes:
        bin_str = ''
        colors = tube.split('-')
        for color in colors:
            bin_str += color_code[color]
        bins.append(bin_str)
        
    # add empty bins
    bins.append('')
    bins.append('')
        
    return bins

In [4]:
def is_bin_complete(b):
    if len(b) != max_vol and len(b) != 0:
        return False
    for i in range(1, len(b)):
        if b[i] != b[0]:
            return False
    return True

In [5]:
def check_win(bins):
    for b in bins:
        if len(b) > 0 and len(b) < max_vol:
            return False
        if not is_bin_complete(b):
            return False
    return True

In [6]:
def is_valid_move(bins, origin, dest):
    # origin and dest should be integer indices of [bins]

    # check to see if there are any balls in origin:
    if len(bins[origin]) <= 0:
        return False
    
    # check to see if there's room in dest
    if len(bins[dest]) >= max_vol:
        return False
    
    # check to see if last ball in dest is same color as last ball in origin
    # or dest is empty
    if len(bins[dest]) > 0 and bins[origin][-1] != bins[dest][-1]:
        return False
    
    # check to make sure [origin] and [dest] are not equal
    if origin == dest:
        return False
    
    # make sure origin and dest are within bounds
    if origin < 0 or dest < 0 or origin >= len(bins) or dest >= len(bins):
        return False
    
    # don't allow moves to remove a ball from a completed bin
    if is_bin_complete(bins[origin]):
        return False
    
    return True

In [7]:
def all_one_color(tube, color):
    count = 0
    for ball in tube:
        if ball == color:
            count += 1
    if count > 0 and count >= len(tube):
        return True
    return False

def is_good_move(bins, origin, dest):
    # do not move ball if all of the same color won't fit in the dest
    color = bins[origin][-1]
    count = 0
    i = len(bins[origin]) - 1
    while i >= 0 and bins[origin][i] == color:
        count += 1
        i -= 1
    if max_vol - len(bins[dest]) < count:
        return False
    
    # if ball in origin can be used to finish a tube other than dest, don't move anywhere else
    if not all_one_color(bins[dest], bins[origin][-1]):
        for i in range(0, len(bins)):
            if i != origin and i != dest:
                if all_one_color(bins[i], bins[origin][-1]):
                    return False
    
    return True

In [8]:
def get_options(bins):
    options = []
    for o in range(0, len(bins)):
        for d in range(0, len(bins)):
            if is_valid_move(bins, o, d) and is_good_move(bins, o, d):
                options.append((o, d))
                
    # randomize options order
    rand_options = []
    while len(options) > 0:
        i = random.randint(0, len(options)-1)
        rand_options.append(options.pop(i))
    
    return rand_options

In [9]:
def process_move(bins, move):
    origin = move[0]
    dest = move[1]
    
    new_bins = []
    for b in bins:
        new_bins.append(b)
    new_bins[dest] = new_bins[dest] + new_bins[origin][-1]
    new_bins[origin] = new_bins[origin][0:-1]
    
    return new_bins

In [10]:
def state_enc(state):
    state_str = ''
    for tube in state:
        state_str += tube + (max_vol-len(tube)) * '*'
    return state_str

def state_unenc(state_str):
    state = []
    while len(state_str) > 0:
        tube = state_str[0:4]
        tube = tube.replace('*', '')
        state.append(tube)
        state_str = state_str[4:]
    return state

In [11]:
def try_options(bins):
    bin_str = state_enc(bins)
    state_dict = {bin_str: []}
    attempted_moves = {bin_str: []}
    
    for i in range(0,100):
        print(f'search depth: {i}')
        new_state_dict = dict()
        
        for state_str in state_dict:
            state = state_unenc(state_str)
            options = get_options(state)
            for option in options:
                # check if move was tried before
                if state_str not in attempted_moves:
                    attempted_moves[state_str] = []
                if option not in attempted_moves[state_str]:
                    attempted_moves[state_str].append(option)
    
                    # check to see if state was reached before
                    new_state = process_move(state, option)
                    new_state_str = state_enc(new_state)
                    if new_state_str not in new_state_dict:
                        new_state_dict[new_state_str] = state_dict[state_str] + [option]
                        if check_win(new_state):
                            return new_state_dict[new_state_str]

        state_dict = dict()
        for new_state_str in new_state_dict:
            state_dict[new_state_str] = new_state_dict[new_state_str]
            
#         print(state_dict)
    return 'solution not found'